In [ ]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

In [ ]:
cus = pd.read_csv("/content/drive/MyDrive/NH/DataSet/cus_info.csv")
iem = pd.read_csv("/content/drive/MyDrive/NH/DataSet/iem_info_20210902.csv")
hist = pd.read_csv("/content/drive/MyDrive/NH/DataSet/stk_bnc_hist.csv")
train_new = pd.read_excel("/content/drive/MyDrive/NH/DataSet/train_new_feature.xlsx")
test_new = pd.read_excel("/content/drive/MyDrive/NH/DataSet/test_new_feature.xlsx")
train_label = pd.read_csv("/content/drive/MyDrive/NH/DataSet/new_train_label.csv")
submission = pd.read_csv("/content/drive/MyDrive/NH/DataSet/sample_submission.csv")

In [ ]:
train=pd.read_csv("/content/drive/MyDrive/NH/DataSet/new_train_data.csv")
test=pd.read_csv("/content/drive/MyDrive/NH/DataSet/new_test_data.csv")

In [ ]:
train_new.head()

In [ ]:
train_new.shape

(31377, 26)

In [ ]:
test_new.head()

In [ ]:
test_new.shape

(70596, 26)

In [ ]:
train_label.shape

(31377, 1)

In [ ]:
train.head()

,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,bnc_qty,tot_aet_amt,stk_par_pr,stk_p
0,36,1,6,0,3,2,5,5,6,9,3037,2,2,1,1.0,69400.0,5000.0,69400.0
1,11,1,6,0,3,2,5,5,6,9,2848,6,1,1,4.0,345200.0,5000.0,86300.0
2,21,1,6,0,3,2,5,5,6,9,974,10,3,99,25.0,66125.0,1000.0,2645.0
3,23,1,6,0,3,2,5,5,6,9,1065,10,2,99,35.0,338800.0,500.0,9680.0
4,23,1,6,0,3,2,5,5,6,9,509,6,2,1,2.0,77700.0,5000.0,38850.0


In [ ]:
train['profit']=train_new['수익률']
train['hold_d']=train_new['보유기간(일)']
train['last_price']=train_new['종가_0630']

In [ ]:
test['profit']=test_new['수익률']
test['hold_d']=test_new['보유기간(일)']
test['last_price']=test_new['종가_1230']

In [ ]:
train.shape

(31377, 21)

In [ ]:
test.shape

(70596, 21)

## NAN 값 처리

In [ ]:
train.isnull().sum()

hist_d                   0
sex_dit_cd               0
cus_age_stn_cd           0
ivs_icn_cd               0
cus_aet_stn_cd           0
mrz_pdt_tp_sgm_cd        0
lsg_sgm_cd               0
tco_cus_grd_cd           0
tot_ivs_te_sgm_cd        0
mrz_btp_dit_cd           0
iem_krl_nm               0
btp_cfc_cd               0
mkt_pr_tal_scl_tp_cd     0
stk_dit_cd               0
bnc_qty                  0
tot_aet_amt              0
stk_par_pr               0
stk_p                    0
profit                  20
hold_d                   0
last_price              20
dtype: int64

In [ ]:
a=train['profit'].mean()
b=train['last_price'].mean()

train_1=train.copy()
train_1.fillna({'profit':a, 'last_price':b}, inplace=True)

In [ ]:
test.isnull().sum()

hist_d                    0
sex_dit_cd                0
cus_age_stn_cd            0
ivs_icn_cd                0
cus_aet_stn_cd            0
mrz_pdt_tp_sgm_cd         0
lsg_sgm_cd                0
tco_cus_grd_cd            0
tot_ivs_te_sgm_cd         0
mrz_btp_dit_cd            0
iem_krl_nm                0
btp_cfc_cd                0
mkt_pr_tal_scl_tp_cd      0
stk_dit_cd                0
bnc_qty                   0
tot_aet_amt               0
stk_par_pr                0
stk_p                     0
profit                  549
hold_d                    0
last_price              549
dtype: int64

In [ ]:
a1=test['profit'].mean()
b1=test['last_price'].mean()

test_1=test.copy()
test_1.fillna({'profit':a1, 'last_price':b1}, inplace=True)

## Feature Selection (콘다 파일 기준)

In [ ]:
train_selected=train_1[{'hist_d', 'tot_aet_amt', 'hold_d', 'last_price'}]
test_selected=test_1[{'hist_d', 'tot_aet_amt', 'hold_d', 'last_price'}]

## 이상치 처리

In [ ]:
def outliers_iqr(data):
  q1, q3= np.percentile(data, [25,75])
  iqr=q3-q1
  lower_bound=q1-(iqr*5)
  upper_bound=q3+(iqr*5)

  return np.where((data>upper_bound)|(data<lower_bound))

In [ ]:
hist_d_outlier=outliers_iqr(train_selected['hist_d'])[0]
tot_aet_amt_outlier=outliers_iqr(train_selected['tot_aet_amt'])[0]
hold_d_outlier=outliers_iqr(train_selected['hold_d'])[0]
last_price_outlier=outliers_iqr(train_selected['last_price'])[0]

outlier_index = np.concatenate((hist_d_outlier, tot_aet_amt_outlier, hold_d_outlier,
                                last_price_outlier), axis=None)

print(len(outlier_index))

4322


In [ ]:
not_oulier_index=[]

for i in train_selected.index:
  if i not in outlier_index:
    not_oulier_index.append(i)

train_clean=train_selected.loc[not_oulier_index]
train_clean=train_selected.reset_index(drop=True)
train_clean

,tot_aet_amt,hold_d,hist_d,last_price
0,69400.0,83,36,62800.0
1,345200.0,110,11,87400.0
2,66125.0,31,21,2540.0
3,338800.0,63,23,9420.0
4,77700.0,55,23,37900.0
...,...,...,...,...
31372,24723500.0,18,16,100500.0
31373,38100.0,262,261,15850.0
31374,183000.0,37,16,85800.0
31375,386500.0,24,17,335000.0


## 정규화

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

scaler.fit(train_clean)
train_scaled=scaler.transform(train_clean)

In [ ]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(train_scaled, train_label, test_size=0.3,shuffle=True, random_state=42)

In [ ]:
lgbm_reg=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=123, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)
gbm_reg=GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                           init=None, learning_rate=0.1, loss='ls', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0, warm_start=False)
rf_reg=RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=123, verbose=0, warm_start=False)

lgbm_reg.fit(X_train, y_train)
gbm_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=-1, oob_score=False,
                      random_state=123, verbose=0, warm_start=False)

In [ ]:
lgbm_pred=lgbm_reg.predict(X_test)
gbm_pred=gbm_reg.predict(X_test)
rf_pred=rf_reg.predict(X_test)

lgbm_rmse=np.sqrt(mse(y_test, lgbm_pred))
gbm_rmse=np.sqrt(mse(y_test, gbm_pred))
rf_rmse=np.sqrt(mse(y_test, rf_pred))

print(lgbm_rmse)
print(gbm_rmse)
print(rf_rmse)

3.693182389064145
0.9651736299783339
1.2050374667897727


In [ ]:
lgbm_real_predict=lgbm_reg.predict(test_selected)
gbm_real_predict=gbm_reg.predict(test_selected)
rf_real_predict=rf_reg.predict(test_selected)

In [ ]:
submission['hold_d']=np.round(lgbm_real_predict)
submission.to_csv("NH_submission_lgbm.csv", index = False)

In [ ]:
submission['hold_d']=np.round(lgbm_real_predict)
submission.to_csv("NH_submission_gbm.csv", index = False)

In [ ]:
submission['hold_d']=np.round(lgbm_real_predict)
submission.to_csv("NH_submission_rf.csv", index = False)